In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def SuperLayer(in_dim, out_dim, width_factor=5, stride=1, blocks=3, dropout=0.3):
    layers = []
    out = out_dim * width_factor
    layers.append(CNNBlock(in_dim, out, stride=stride, dropout=dropout))
    for i in range(blocks-1):
        layers.append(CNNBlock(out, out, dropout=dropout))
    
    return layers

class CNNBlock(nn.Module):
    def __init__(self, in_dim, out_dim, stride=1, dropout=0.):
        super().__init__()
        layers = [
            nn.BatchNorm2d(in_dim),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_dim, out_dim, 3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Conv2d(out_dim, out_dim, 3, stride=1, padding=1, bias=False)
        ]
        self.conv_block = nn.Sequential(*layers)
        self.residual = nn.Sequential(nn.Conv2d(in_dim, out_dim, 1, stride=stride, padding=0, bias=False))

    def forward(self, x): return self.residual(x) + self.conv_block(x)

class WideResNet(nn.Module):
    def __init__(self, input_shape, classes):
        super().__init__()
        layer1 = SuperLayer(3, 8)
        layer2 = SuperLayer(40, 16, stride=2,)
        layer3 = SuperLayer(80, 32, stride=2)
        self.layers = nn.Sequential(*layer1, *layer2, *layer3, nn.BatchNorm2d(160), nn.ReLU(inplace=True), nn.Flatten())
        with torch.no_grad():
            s = self.layers(torch.rand(1, *input_shape)).shape
            print(s)
            n_flatten = s[1]
        self.fc = nn.Linear(n_flatten, classes)

    def forward(self, x):
        return self.fc(self.layers(x))



In [2]:
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
import os
from sklearn.model_selection import train_test_split

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)#[1:]
        self.file_paths = []
        self.labels = []
        for i, class_ in enumerate(self.classes):
            
            class_dir = os.path.join(root_dir, class_)
            files = os.listdir(class_dir)#[1:]
            self.file_paths += [os.path.join(class_dir, file) for file in files]
            self.labels += [i] * len(files)

        #self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)
        #return image.to(self.device), torch.tensor(label, device=self.device)

# Define your transformations
transform = transforms.Compose([
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize your dataset
dataset = CustomImageDataset(root_dir='./down/data/', transform=transform)
train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=0.25)

train_data = Subset(dataset, train_idx)
test_data = Subset(dataset, val_idx)


# Initialize your dataloader
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)
print(len(dataset))


5125


In [3]:
model = WideResNet(dataset.__getitem__(1)[0].shape, 5)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
losses = []
val_accuracies = []
# Number of epochs
epochs = 10

# Training loop
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        log_freq = 16
        if ((i+1) % log_freq) == 0:    # print every 4 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / log_freq))
            losses.append(running_loss)
            running_loss = 0.0
    correct = 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # forward + backward + optimize
            outputs = model(inputs)
            pred = torch.argmax(outputs, dim=1)

            right = pred == labels
            correct += torch.sum(right).item()
    model.train()
    test_accuracy = correct/len(test_data)
    val_accuracies.append(test_accuracy)

    print("Epoch: {}, test acc: {:.4f}\n".format(epoch, test_accuracy))

print('Finished Training')

torch.Size([1, 1656000])
[1,    16] loss: 135.248
[1,    32] loss: 55.821
[1,    48] loss: 54.950
[1,    64] loss: 41.646
[1,    80] loss: 44.068
[1,    96] loss: 37.707
[1,   112] loss: 32.965
[1,   128] loss: 44.114
[1,   144] loss: 44.867
[1,   160] loss: 39.037
[1,   176] loss: 22.449
[1,   192] loss: 26.937
[1,   208] loss: 27.304
[1,   224] loss: 26.845
[1,   240] loss: 21.755
Epoch: 0, test acc: 0.6047

[2,    16] loss: 12.452
[2,    32] loss: 4.769
[2,    48] loss: 12.822
[2,    64] loss: 13.110
[2,    80] loss: 5.584
[2,    96] loss: 8.328
[2,   112] loss: 9.012
[2,   128] loss: 7.698
[2,   144] loss: 8.327
[2,   160] loss: 5.746
[2,   176] loss: 3.105
[2,   192] loss: 5.763
[2,   208] loss: 5.184
[2,   224] loss: 5.906
[2,   240] loss: 4.672
Epoch: 1, test acc: 0.7008

[3,    16] loss: 3.038
[3,    32] loss: 3.933
[3,    48] loss: 7.036
[3,    64] loss: 4.262
[3,    80] loss: 9.039
[3,    96] loss: 7.125
[3,   112] loss: 6.224
[3,   128] loss: 5.427
[3,   144] loss: 5.505
[3,

In [5]:
np_losses = np.array(losses)
np_accs = np.array(val_accuracies)
np.save("wres_losses", np_losses)
np.save("wres_accs", np_accs)

In [6]:
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)
model.eval()
correct = 0
with torch.no_grad():
    for i, data in enumerate(test_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # forward + backward + optimize
        outputs = model(inputs)
        pred = torch.argmax(outputs, dim=1)

        right = pred == labels
        correct += torch.sum(right).item()

print("Accuracy: " + str(correct/len(test_data)))
print(len(test_data))

Accuracy: 0.94140625
768


In [6]:
torch.save(model.state_dict(), "./models/5120wideresnet.pt")

In [ ]:
######################################################################################################################

In [3]:
class ConvNet(nn.Module):

  def __init__(self):
      super().__init__()

      self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size=(5, 5), stride=2, padding=1)
      self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 12, kernel_size=(5, 5), stride=2, padding=1)
      self.conv3 = nn.Conv2d(in_channels = 12, out_channels = 16, kernel_size=(3, 3), padding=1)
      self.flat = nn.Flatten()
      with torch.no_grad():
          n_flatten = self.flat(self.conv3(self.conv2(self.conv1(torch.rand(1, *train_data.__getitem__(0)[0].shape))))).shape
          print(n_flatten)
          n_flatten = n_flatten[1]

      self.fc1 = nn.Linear(in_features=n_flatten, out_features=500)
      print(self.fc1)
      self.fc2 = nn.Linear(in_features=500, out_features=50)
      self.fc3 = nn.Linear(in_features=50, out_features=4)

  def forward(self, X):

      X = F.relu(self.conv1(X))
      #X = F.max_pool2d(X, 2)

      X = F.relu(self.conv2(X))
      #X = F.max_pool2d(X, 2)

      X = F.relu(self.conv3(X))
      #X = F.max_pool2d(X, 2)
      X = self.flat(X)

      X = F.relu(self.fc1(X))
      X = F.relu(self.fc2(X))
      X = self.fc3(X)

      return X


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNet()
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
losses = []
val_accuracies = []
# Number of epochs
epochs = 10

# Training loop
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        log_freq = 16
        if ((i+1) % log_freq) == 0:    # print every 16 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / log_freq))
            losses.append(running_loss / log_freq)
            running_loss = 0.0
    correct = 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # forward + backward + optimize
            outputs = model(inputs)
            pred = torch.argmax(outputs, dim=1)

            right = pred == labels
            correct += torch.sum(right).item()
    model.train()
    test_accuracy = correct/len(test_data)
    val_accuracies.append(test_accuracy)

    print("Epoch: {}, test acc: {:.4f}\n".format(epoch, test_accuracy))

print('Finished Training')

torch.Size([1, 162112])
Linear(in_features=162112, out_features=500, bias=True)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [35]:
torch.save(model.state_dict(), "./models/lightcnn.pt")